# CIFAR10 이미지 데이터 학습

In [2]:
from tensorflow import keras

(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.cifar10.load_data()

X_train = X_train_full[5000:]
y_train = y_train_full[5000:]
X_valid = X_train_full[:5000]
y_valid = y_train_full[:5000]

In [5]:
X_train.shape, y_train.shape

((45000, 32, 32, 3), (45000, 1))

In [61]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
'''for i in range(20):
    model.add(keras.layers.Dense(100, activation="elu",
                                kernel_initializer="he_normal"))'''

'''for i in range(20):
    model.add(keras.layers.Dense(100, kernel_initializer="he_normal"))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Activation("elu"))'''

for i in range(15):
    model.add(keras.layers.Dense(100, activation="selu",
                                 kernel_initializer="lecun_normal"))
model.add(keras.layers.AlphaDropout(rate=0.1))
X_means = X_train.mean(axis=0)
X_stds = X_train.std(axis=0)
X_train_scaled = (X_train - X_means) / X_stds
X_valid_scaled = (X_valid - X_means) / X_stds
X_test_scaled = (X_test - X_means) / X_stds
    
model.add(keras.layers.Dense(10, activation="softmax"))
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_8 (Flatten)          (None, 3072)              0         
_________________________________________________________________
dense_153 (Dense)            (None, 100)               307300    
_________________________________________________________________
dense_154 (Dense)            (None, 100)               10100     
_________________________________________________________________
dense_155 (Dense)            (None, 100)               10100     
_________________________________________________________________
dense_156 (Dense)            (None, 100)               10100     
_________________________________________________________________
dense_157 (Dense)            (None, 100)               10100     
_________________________________________________________________
dense_158 (Dense)            (None, 100)              

In [62]:
opt = keras.optimizers.Nadam(lr=5e-5)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=opt,
              metrics=["accuracy"])

In [63]:
import os

early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_cifar10_model.h5", save_best_only=True)
run_index = 9 # increment every time you train the model
run_logdir = os.path.join(os.curdir, "my_cifar10_logs", "run_{:03d}".format(run_index))
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
callbacks = [early_stopping_cb, model_checkpoint_cb, tensorboard_cb]

In [29]:
'''model.fit(X_train, y_train, epochs=100,
          validation_data=(X_valid, y_valid),
          callbacks=callbacks)'''

model.fit(X_train_scaled, y_train, epochs=100,
          validation_data=(X_valid_scaled, y_valid),
          callbacks=callbacks)

Epoch 1/100
1407/1407 [==============================] - 15s 9ms/step - loss: 2.1966 - accuracy: 0.2383 - val_loss: 1.7506 - val_accuracy: 0.3852
Epoch 2/100
1407/1407 [==============================] - 10s 7ms/step - loss: 1.7622 - accuracy: 0.3773 - val_loss: 1.6326 - val_accuracy: 0.4166
Epoch 3/100
1407/1407 [==============================] - 10s 7ms/step - loss: 1.6201 - accuracy: 0.4257 - val_loss: 1.5582 - val_accuracy: 0.4468
Epoch 4/100
1407/1407 [==============================] - 10s 7ms/step - loss: 1.5087 - accuracy: 0.4640 - val_loss: 1.5304 - val_accuracy: 0.4616
Epoch 5/100
1407/1407 [==============================] - 9s 7ms/step - loss: 1.4600 - accuracy: 0.4837 - val_loss: 1.5527 - val_accuracy: 0.4736
Epoch 6/100
1407/1407 [==============================] - 10s 7ms/step - loss: 1.3867 - accuracy: 0.5068 - val_loss: 1.5350 - val_accuracy: 0.4688
Epoch 7/100
1407/1407 [==============================] - 9s 7ms/step - loss: 1.3358 - accuracy: 0.5250 - val_loss: 1.5223 - v

---

In [30]:
class MCAlphaDropout(keras.layers.AlphaDropout):
    def call(self, inputs):
        return super().call(inputs, training=True)
    
mc_model = keras.models.Sequential([
    MCAlphaDropout(layer.rate) if isinstance(layer, keras.layers.AlphaDropout) else layer
    for layer in model.layers
])

def mc_dropout_predict_probas(mc_model, X, n_samples=10):
    Y_probas = [mc_model.predict(X) for sample in range(n_samples)]
    return np.mean(Y_probas, axis=0)

def mc_dropout_predict_classes(mc_model, X, n_samples=10):
    Y_probas = mc_dropout_predict_probas(mc_model, X, n_samples)
    return np.argmax(Y_probas, axis=1)

In [32]:
import numpy as np
y_pred = mc_dropout_predict_classes(mc_model, X_valid_scaled)
accuracy = np.mean(y_pred == y_valid[:, 0])
accuracy

0.4876

In [42]:
K = keras.backend

class OneCycleScheduler(keras.callbacks.Callback):
    def __init__(self, iterations, max_rate, start_rate=None,
                 last_iterations=None, last_rate=None):
        self.iterations = iterations
        self.max_rate = max_rate
        self.start_rate = start_rate or max_rate / 10
        self.last_iterations = last_iterations or iterations // 10 + 1
        self.half_iteration = (iterations - self.last_iterations) // 2
        self.last_rate = last_rate or self.start_rate / 1000
        self.iteration = 0
    def _interpolate(self, iter1, iter2, rate1, rate2):
        return ((rate2 - rate1) * (self.iteration - iter1)
                / (iter2 - iter1) + rate1)
    def on_batch_begin(self, batch, logs):
        if self.iteration < self.half_iteration:
            rate = self._interpolate(0, self.half_iteration, self.start_rate, self.max_rate)
        elif self.iteration < 2 * self.half_iteration:
            rate = self._interpolate(self.half_iteration, 2 * self.half_iteration,
                                     self.max_rate, self.start_rate)
        else:
            rate = self._interpolate(2 * self.half_iteration, self.iterations,
                                     self.start_rate, self.last_rate)
        self.iteration += 1
        K.set_value(self.model.optimizer.lr, rate)

In [64]:
import math 

n_epochs = 100
onecycle = OneCycleScheduler(math.ceil(len(X_train_scaled) / batch_size) * n_epochs, max_rate=0.05)
history = model.fit(X_train_scaled, y_train, epochs=n_epochs, batch_size=batch_size,
                    validation_data=(X_valid_scaled, y_valid),
                    callbacks=[tensorboard_cb, early_stopping_cb, model_checkpoint_cb])

Epoch 1/100
352/352 [==============================] - 9s 17ms/step - loss: 2.3370 - accuracy: 0.2238 - val_loss: 1.7682 - val_accuracy: 0.3740
Epoch 2/100
352/352 [==============================] - 3s 10ms/step - loss: 1.8665 - accuracy: 0.3505 - val_loss: 1.6747 - val_accuracy: 0.4106
Epoch 3/100
352/352 [==============================] - 3s 10ms/step - loss: 1.7506 - accuracy: 0.3862 - val_loss: 1.6220 - val_accuracy: 0.4276
Epoch 4/100
352/352 [==============================] - 3s 9ms/step - loss: 1.6477 - accuracy: 0.4194 - val_loss: 1.5825 - val_accuracy: 0.4436
Epoch 5/100
352/352 [==============================] - 3s 9ms/step - loss: 1.5746 - accuracy: 0.4467 - val_loss: 1.5688 - val_accuracy: 0.4592
Epoch 6/100
352/352 [==============================] - 3s 9ms/step - loss: 1.5122 - accuracy: 0.4670 - val_loss: 1.5556 - val_accuracy: 0.4644
Epoch 7/100
352/352 [==============================] - 3s 9ms/step - loss: 1.4517 - accuracy: 0.4870 - val_loss: 1.5556 - val_accuracy: 0.4